In [1]:
import pandas as pd
from datetime import datetime

### Read zoning regulations
The CSV file is generated by the `CombineJurisdictions.ipynb` notebook in this repository.

In [2]:
zoning = pd.read_csv(
    './final.csv',
    na_values=[' ']
)

### Define a function to clean up minimum lot size into categories

In [3]:
def clean_min_lot_size(x):
    x = str(x).split(' ')[0].split(',')[0].split('-')[0]
    x = float(x)
    
    if x >= 1.84:
        return "1.84 or more"
    
    if x >= 0.92:
        return "0.92 or more"
    
    if x >= 0.46:
        return "0.46 or more"
    
    return "under 0.46"
    

zoning['1F MinLotSize'] = zoning['1-Family Min. Lot (ACRES)'].apply(clean_min_lot_size)
zoning['1F MinLotSize'].value_counts()

under 0.46      1714
0.92 or more     281
1.84 or more     260
0.46 or more     242
Name: 1F MinLotSize, dtype: int64

### Define functions to calculate acre values per given column

In [5]:
c_juris = 'Jurisdiction'

def all_acres():
    return pd.DataFrame(
        zoning.groupby([c_juris])['MunicipalAcres'].sum()
    )

# When acres per each individual value in column needs to be calculated 
def calc_acres(col):
    
    df = zoning.groupby([c_juris, col])['MunicipalAcres'].sum().reset_index()\
        .pivot(index=c_juris, columns=col, values='MunicipalAcres')

    df.columns = [ f"{df.columns.name} {x}" for x in df.columns]

    return df


# When we only need to determine if for a given zone
# any value across `cols` is set or not set
def any_value_acres(cols):
    
    # The first item in the list is how we should call the stats column
    col_name = cols[0]
    
    # Make sure we divide by N-family Allowed/Conditional acres
    # (for Any-family column, divide by 1-family)
    fam_col =  f'{cols[1][0]}-Family' if cols[1][0] != '4' else '4+-Family'
    
    df = zoning.filter([c_juris, 'MunicipalAcres'] + cols[1:] + [fam_col])
    
    df[col_name] = df.filter(cols[1:]).any(axis=1)  
    
    df[col_name] = df.apply(
        lambda row: 'AnyValue' if row[col_name] and row[fam_col] in ['Allowed/Conditional', 'Special Permit']
            else 'NoValue', axis=1
    )
    
    df = df.groupby([c_juris, col_name])['MunicipalAcres'].sum().reset_index()\
        .pivot(index=c_juris, columns=col_name, values='MunicipalAcres')
    
    df.columns = [ f"{df.columns.name}_{x}" for x in df.columns]

    return df

In [6]:
# Columns for which unique value counts should be aggregated
# Eg for `1-Family`, would create following columns:
#     1-Family Allowed/Conditional
#     1-Family Prohibited
#     1-Family Special Permit
value_sums = [
    '1-Family',
    '2-Family',
    '3-Family',
    '4+-Family',
    'Elderly Housing District',
    '1F MinLotSize'
]

# A list of aggregations
# First value in nested list - name of new aggregation (eg 'MinLotSize1F')
# All consecutive values in nexted list - names of columns which should be
# searched for _any_ non-null value
any_value_sums = [
    
    ### Minimum Lot Size ###
    
    ['MinLotSize1F', # 1-family
         '1-Family Min. Lot (ACRES)'
    ],
    
    ['MinLotSize2F', # 2-family
         '2-Family Min. Lot (ACRES)'
    ],
    
    ['MinLotSize3F', # 3-family
         '3-Family Min. Lot (ACRES)'
    ],
    
    ['MinLotSize4F', # 4+ family
         '4+-Family Min. Lot (ACRES)'
    ],
    
    ['MinLotSize', # any size
         '1-Family Min. Lot (ACRES)',
         '2-Family Min. Lot (ACRES)',
         '3-Family Min. Lot (ACRES)',
         '4+-Family Min. Lot (ACRES)'
    ],
    
    
    ### Minimum Unit Size ###
    
    ['MinUnitSize1F', # single-family
         '1-Family Min. Floor Area for the housing unit itself (not the lot) (SF)'
    ],
    
    ['MinUnitSize2F', # 2-family
         '2-Family Min. Floor Area for each housing unit (not lot size) (SF)'
    ],
    
    ['MinUnitSize3F', # 3-family
         '3-Family Min. Floor Area for each housing unit (not lot size) (SF)'
    ],
    
    ['MinUnitSize4F', # 4+ family
         '4+-Family Min. Floor Area per housing unit (not lot size) (SF)'
    ],
    
    ['MinUnitSize', # any size
         '1-Family Min. Floor Area for the housing unit itself (not the lot) (SF)',
         '2-Family Min. Floor Area for each housing unit (not lot size) (SF)',
         '3-Family Min. Floor Area for each housing unit (not lot size) (SF)',
         '4+-Family Min. Floor Area per housing unit (not lot size) (SF)'
    ],
    
    
    ### Floor-to-Area Ratio (FAR) ###
    
    ['FloorToArea1F', # 1-family
         '1-Family Floor to Area Ratio'
    ],
    
    ['FloorToArea2F', # 2-family
         '2-Family Floor to Area Ratio'
    ],
    
    ['FloorToArea3F', # 3-family
         '3-Family Floor to Area Ratio'
    ],
    
    ['FloorToArea4F', # 4+ family
         '4-Family Floor to Area Ratio'
    ],
    
    ['FloorToArea', # any size
         '1-Family Floor to Area Ratio',
         '2-Family Floor to Area Ratio',
         '3-Family Floor to Area Ratio',
         '4-Family Floor to Area Ratio'
    ],

    
    ### Parking ###
    
    ['Parking1F', # 1-family
         '1-Family Min. # Parking Spaces'
    ],
    
    ['Parking2F', # 2-family
         '2-Family Min. # Parking Spaces Per Studio or 1BR',
         '2-Family Min. # Parking Spaces Per 2+ BR'
    ],
    
    ['Parking3F', # 3-family
         '3-Family Min. # Parking Spaces Per Studio or 1BR',
         '3-Family Min. # Parking Spaces Per 2+ BR'
    ],
    
    ['Parking4F', # 4+-family
         '4+-Family Min. # Parking Spaces Per Studio or 1BR',
         '4+-Family Min. # Parking Spaces Per 2+ BR'
    ],
    
    ['Parking', # any family
         '1-Family Min. # Parking Spaces',
         '2-Family Min. # Parking Spaces Per Studio or 1BR',
         '2-Family Min. # Parking Spaces Per 2+ BR',
         '3-Family Min. # Parking Spaces Per Studio or 1BR',
         '3-Family Min. # Parking Spaces Per 2+ BR',
         '4+-Family Min. # Parking Spaces Per Studio or 1BR',
         '4+-Family Min. # Parking Spaces Per 2+ BR'
    ],
    
    ### Maximum Height ###
    
    ['MaxHeight1F', # 1-family
         '1-Family Max. Height (# of stories)',
         '1-Family Max. Height (# of feet)'
    ],
    
    ['MaxHeight2F', # 2-family
         '2-Family Max. Height (# of stories)',
         '2-Family Max. Height (# of feet)'
    ],
    
    ['MaxHeight3F', # 3-family
         '3-Family Max. Height (# of stories)',
         '3-Family Max. Height (# of feet)'
    ],
    
    ['MaxHeight4F', # 4+-family
         '4-Family Max. Height (# of stories)',
         '4-Family Max. Height (# of feet)'
    ],
    

    ['MaxHeight', # any family
         '1-Family Max. Height (# of stories)',
         '1-Family Max. Height (# of feet)',
         '2-Family Max. Height (# of stories)',
         '2-Family Max. Height (# of feet)',
         '3-Family Max. Height (# of stories)',
         '3-Family Max. Height (# of feet)',
         '4-Family Max. Height (# of stories)',
         '4-Family Max. Height (# of feet)'
    ],
    
    
    ### Lot Coverage ###
    
    ['LotCoverage1F', # 1-family
         '1-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '1-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)'
    ],
    
    ['LotCoverage2F', # 2-family
         '2-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '2-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)'
    ],
    
    ['LotCoverage3F', # 3-family
         '3-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '3-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)'
    ],
    
    ['LotCoverage4F', # 4+-family
         '4-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '4-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)'
    ],
    

    ['LotCoverage', # any family
         '1-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '1-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)',
         '2-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '2-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)',
         '3-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '3-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)',
         '4-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS (%)',
         '4-Family Max. Lot Coverage IMPERVIOUS - BUILDINGS & PAVEMENT (%)'
    ],
    
    
    ### Density (Units/Acre) Cap ###
    
    ['MaxDensity2F', # 2-family
        '2-Family Max. Density (UNITS/ACRE)'
    ],
    
    ['MaxDensity3F', # 3-family
        '3-Family Max. Density (UNITS/ACRE)'
    ],
    
    ['MaxDensity4F', # 4+ family
         '4+-Family Max. Density (UNITS/ACRE)'
        
    ],
    
    ['MaxDensity', # any family
         '2-Family Max. Density (UNITS/ACRE)',
         '3-Family Max. Density (UNITS/ACRE)',
         '4+-Family Max. Density (UNITS/ACRE)'
    ]
    
]

final = pd.concat(
    [ all_acres() ]
        + [ calc_acres(c) for c in value_sums ]
        + [ any_value_acres(cs) for cs in any_value_sums ],
    axis=1
)

### Save stats as acres

In [7]:
final.fillna(0).drop(columns=[
    '1-Family Overlay',
    '2-Family Overlay',
    '3-Family Overlay',
    '4+-Family Overlay',
]).filter([ x for x in final.columns if not x.endswith('NoValue')])\
    .to_csv(f"./juris-stats-{datetime.now().strftime('%Y-%m-%d-%H-%M')}.csv", float_format='%.0f')

### Save stats as percentages

In [8]:
final_perc = final.fillna(0).drop(columns=[
    '1-Family Overlay',
    '2-Family Overlay',
    '3-Family Overlay',
    '4+-Family Overlay',
])

final_perc.loc[:, '1-Family Allowed/Conditional':] = final_perc.loc[:, '1-Family Allowed/Conditional':]\
    .div(final_perc.MunicipalAcres, axis='rows') * 100

final_perc.filter([ x for x in final_perc.columns if not x.endswith('NoValue')])\
    .to_csv(f"./juris-stats-perc-{datetime.now().strftime('%Y-%m-%d-%H-%M')}.csv", float_format='%.1f')